In [5]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Define the data as a Pandas DataFrame
data = {
    'id': [1, 2],

    'product_des': [
        [
        {'unit': 'origin', 'value': 32, 'dimentionality': 1},
        {'unit': 'inches', 'value': 452, 'dimentionality': 2}
    
        
    ],
    [
        {'unit': 'ounches', 'value': 32, 'dimentionality': 1},
        {'unit': 'inches', 'value': 452, 'dimentionality': 2} 
    ]
    ]
}
df = pd.DataFrame(data)

# Define the schema using PyArrow
schema = pa.schema([
    pa.field('id', pa.int64()),
    pa.field('product_des', pa.list_(pa.struct([
        pa.field('unit', pa.string()),
        pa.field('value', pa.int64()),
        pa.field('dimentionality', pa.int64())
        ])))

  
])

# Convert the Pandas DataFrame to PyArrow Table
table = pa.Table.from_pandas(df, schema=schema)

# Write the PyArrow Table to Parquet file
# with pq.ParquetWriter(r'C:\Users\User\PycharmProjects\pythonProject3\Planet_X\Phase_1', schema) as writer:
#     writer.write_table(table)


In [12]:
pq.write_table(table, r"C:\Users\User\PycharmProjects\pythonProject3\Planet_X\Phase_1\exam.parquet")

In [13]:
table = pq.read_table('exam.parquet')

# Convert the table to a Pandas DataFrame
df = table.to_pandas()

In [14]:
print(df)

   id                                        product_des
0   1  [{'unit': 'origin', 'value': 32, 'dimentionali...
1   2  [{'unit': 'ounches', 'value': 32, 'dimentional...


In [22]:
import pandas as pd
import pyarrow as pa

# Define some example data
data = [
    {'id': 1, 'product_des': [{'unit': 'origin', 'value': 32, 'dimentionality': 1}, {'unit': 'inches', 'value': 452, 'dimentionality': 2}]},
    {'id': 2, 'product_des': [{'unit': 'ounces', 'value': 32, 'dimentionality': 1}, {'unit': 'inches', 'value': 452, 'dimentionality': 2}, {'unit': 'cm', 'value': 10, 'dimentionality': 1}]},
    {'id': 3, 'product_des': [{'unit': 'meters', 'value': 2, 'dimentionality': 1}, {'unit': 'pounds', 'value': 5, 'dimentionality': 1}, {'unit': 'inches', 'value': 100, 'dimentionality': 2}, {'unit': 'cm', 'value': 20, 'dimentionality': 1}]},
]

# Create a list of structs for each row
structs = []
for row in data:
    product_des = row['product_des']
    struct_list = []
    for desc in product_des:
        struct_type = pa.struct([
            pa.field('unit', pa.string()),
            pa.field('value', pa.int64()),
            pa.field('dimentionality', pa.int64())
        ])
        struct_list.append(struct_type(
            desc['unit'],
            desc['value'],
            desc['dimentionality']
        ))
    structs.append(pa.array(struct_list))

# Create the PyArrow schema and table
schema = pa.schema([
    pa.field('id', pa.int64()),
    pa.field('product_des', pa.list_(pa.struct([
        pa.field('unit', pa.string()),
        pa.field('value', pa.int64()),
        pa.field('dimentionality', pa.int64())
    ])))
])
table = pa.Table.from_arrays([
    pa.array([row['id'] for row in data]),
    pa.list_(structs)
], schema=schema)

# Write the table to a Parquet file
pq.write_table(table, 'product_descriptions.parquet')

# Read the table back from the Parquet file
table = pq.read_table('product_descriptions.parquet')

# Convert the PyArrow table to a Pandas DataFrame
df = table.to_pandas()
print(df)


TypeError: 'pyarrow.lib.StructType' object is not callable

In [20]:
df = pd.read_parquet('product_attrs.parquet.gzip')

In [33]:
# import os
from country_call import find_country
import pandas as pd
import re
from material_selection import find_materials
# df = pd.read_parquet('product_attrs.parquet.gzip')
df1 = pd.DataFrame(columns=['Product_id', 'Origin', 'Material','inches','inches_dimension','Pounds','Ounces','Lb','Quantity'])
data=[]
id = []
product_des=[]
for index, row in df.head(10).iterrows():
    print(f"index: {index}")
    dimention_1=""
    dimention_2=""
    dimention_3=""
    dimention_4=""
    dimention_5=""
    inches=""
    pounds=""
    ounces=""
    lb=""
    product_id=""
    country=""
    material=[]
    quantity=""
    data=[]
    
    # Finding Dimention 1
    product_id=row[1]
    id.append(product_id)
    if row[2]!=None:
        dimention_unit=row[2].split(" ")[-1]
        if dimention_unit=="inches":
            numbers = re.findall(r'\d+\.\d+|\d+', row[2])
            dimention_1=(len(numbers))
            inches = 1
            for num in numbers:
                inches *= float(num)
            diction={'unit':'inches','value':float(inches),'dimentionality':dimention_1}
            data.append(diction)    
                
     # Finding Dimention 2               
    if row[3]!=None:
        dimention_unit=row[3].split(" ")[-1]
        dimention_unit=dimention_unit.lower()
        if dimention_unit=="pounds":
            numbers = re.findall(r'\d+\.\d+|\d+', row[3])
            dimention_2=1
            pounds=numbers[0]
            diction={'unit':'pounds','value':float(pounds),'dimentionality':1}
            data.append(diction)
            
            
    # Finding Dimention 3            

    if row[3]!=None:
        dimention_unit=row[3].split(" ")[-1]
        dimention_unit=dimention_unit.lower()
        if dimention_unit=="ounces":
            numbers = re.findall(r'\d+\.\d+|\d+', row[3])
            dimention_3=1
            ounces=numbers[0]
            diction={'unit':'ounces','value':float(ounces),'dimentionality':1}
            data.append(diction)

            
    # Finding Dimention 4
    if row[2]!=None:
        dimention_unit=row[2].split("l")[-1]
        dimention_unit=dimention_unit.lower()
        if dimention_unit=="b":
            numbers = re.findall(r'\d+\.\d+|\d+', row[2])
            dimention_4=1
            lb=numbers[0]
            diction={'unit':'lb','value':float(lb),'dimentionality':1}
            data.append(diction)

                
    if row[2]!=None:
        dimention_unit=row[2].split('"')[-1]
        dimention_unit=dimention_unit.lower()
        if dimention_unit=="h" or dimention_unit=="w" or dimention_unit=="l" or dimention_unit=="d":
            numbers = re.findall(r'\d+\.\d+|\d+', row[2])
            dimention_1=(len(numbers))
            inches = 1
            for num in numbers:
                inches *= float(num)
            # print(inches)
            # print(dimention_5)
            diction={'unit':'inches','value':float(inches),'dimentionality':dimention_1}
            data.append(diction)
                
                
                
            
    title=f"Index: {index}, Row data: {row[4]}"
    descrip=f"Index: {index}, Row data: {row[5]}"
    country=find_country(title)
    if country == None:
        country = find_country(descrip)
        diction={'unit':'origin','value':5,'dimentionality':1}
        data.append(diction)
    # print(country)
    material=find_materials(title)
    if material == None:
        material = find_materials(descrip)
        diction={'unit':'materials','value':8,'dimentionality':1}
        data.append(diction)   
    
    # Use regular expressions to find the quantity information
    text=f"{title} {descrip}"
    match = re.search(r"(pack of|pk|pcs|pieces|set|qty|piece|set of|piece of)\s?(\d+)", text, flags=re.IGNORECASE)
    if match==None:
        match = re.search(r"(\d+)(\s?|\s*)(pack|pk|pcs|pieces|set|packs|sets|piece|Pair Pack|Pair Packs|Count|Counts)", text, flags=re.IGNORECASE)
    if match==None:
        match = re.search(r"numeric_(\d+)", text, flags=re.IGNORECASE)
    if match==None:
        match = re.search(r"(\d+)\s*-\s*(pk|pcs|pieces|set|piece|pack|packs|Pair Pack|Pair Packs|Count|Counts)", text, flags=re.IGNORECASE)

    if match:
        quantity = match.group()
        diction={'unit':'quantity','value':6,'dimentionality':1}
        data.append(diction)
    product_des.append(data)
main_data={
    'id':id,
    'product_des':product_des    
}
        # print("Quantity:", quantity)
    # else:
        # print("Quantity not found.")

    # data = {'Product_id':product_id, 'Origin':country, 'Material':str(material),'inches':inches,'inches_dimension':dimention_1,'Pounds':pounds,'Ounces':ounces,'Lb':lb,'Quantity':quantity}
    # df1 = df1.append(data, ignore_index=True)
    
# Create a list of structs for each row
print(main_data)




index: 0
index: 1
index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
{'id': ['078c02969df534dd252ead140ff1e2e2', '5daf53900ecea17db9d855cde97819de', '2f0ee547ae16a9e3ca19e7ba1d1f14e9', 'c74918fa58c77ef27a96e3badd8e7566', '6c234fb210cc667034e1161cba15d8c5', '759d9dd6bc1ac62995bd69db5b626bfd', 'd75a82375ca598bd148cd5061006054c', 'f3d5db1b08f5fb394ff73834238e9e69', 'acc54e85b1677734e5085a34e4b4b0c7', '845b7af056e19a8eb729d907e43a9b42'], 'product_des': [[{'unit': 'origin', 'value': 5, 'dimentionality': 1}], [{'unit': 'inches', 'value': 140.0, 'dimentionality': 3}, {'unit': 'origin', 'value': 5, 'dimentionality': 1}], [{'unit': 'origin', 'value': 5, 'dimentionality': 1}, {'unit': 'quantity', 'value': 6, 'dimentionality': 1}], [{'unit': 'inches', 'value': 140.0, 'dimentionality': 3}, {'unit': 'ounces', 'value': 7.2, 'dimentionality': 1}, {'unit': 'origin', 'value': 5, 'dimentionality': 1}], [{'unit': 'origin', 'value': 5, 'dimentionality': 1}, {'unit': 'quantity', 'value

In [34]:
df1 = pd.DataFrame(main_data)

# Define the schema using PyArrow



In [35]:
schema = pa.schema([
    pa.field('id', pa.string()),
    pa.field('product_des', pa.list_(pa.struct([
        pa.field('unit', pa.string()),
        pa.field('value', pa.float64()),
        pa.field('dimentionality', pa.int64())
        ])))

  
])

# Convert the Pandas DataFrame to PyArrow Table
table = pa.Table.from_pandas(df1, schema=schema)

In [38]:
table

pyarrow.Table
id: string
product_des: list<item: struct<unit: string, value: double, dimentionality: int64>>
  child 0, item: struct<unit: string, value: double, dimentionality: int64>
      child 0, unit: string
      child 1, value: double
      child 2, dimentionality: int64
----
id: [["078c02969df534dd252ead140ff1e2e2","5daf53900ecea17db9d855cde97819de","2f0ee547ae16a9e3ca19e7ba1d1f14e9","c74918fa58c77ef27a96e3badd8e7566","6c234fb210cc667034e1161cba15d8c5","759d9dd6bc1ac62995bd69db5b626bfd","d75a82375ca598bd148cd5061006054c","f3d5db1b08f5fb394ff73834238e9e69","acc54e85b1677734e5085a34e4b4b0c7","845b7af056e19a8eb729d907e43a9b42"]]
product_des: [[    -- is_valid: all not null
    -- child 0 type: string
["origin"]
    -- child 1 type: double
[5]
    -- child 2 type: int64
[1],    -- is_valid: all not null
    -- child 0 type: string
["inches","origin"]
    -- child 1 type: double
[140,5]
    -- child 2 type: int64
[3,1],...,    -- is_valid: all not null
    -- child 0 type: string
["

In [48]:
import pyarrow.compute as pc
target_id='5daf53900ecea17db9d855cde97819de'
filtered_table = table.filter(pc.equal(table['id'], target_id))

In [49]:
filtered_product_des_column = filtered_table['product_des']

In [50]:
filtered_id_column = filtered_table['id']
filtered_product_des_column = filtered_table['product_des']

In [51]:
filtered_id_values = filtered_id_column.to_pandas().tolist()
filtered_product_des_values = filtered_product_des_column.to_pandas().tolist()

In [52]:
print(filtered_id_values)
print(filtered_product_des_values)

['5daf53900ecea17db9d855cde97819de']
[array([{'unit': 'inches', 'value': 140.0, 'dimentionality': 3},
       {'unit': 'origin', 'value': 5.0, 'dimentionality': 1}],
      dtype=object)]
